# Finite Difference Method for Coupon Bond Pricing

---

## PDE Setup and SDE

We begin with a coupon bond whose price is denoted by $ B(r, t) $, where the short rate $ r $ follows a stochastic process:

$$
dr = \kappa(\theta e^{\mu t} - r) \, dt + \sigma r^\beta \, dW
$$

The bond pays continuous coupons $ Ce^{-\alpha t}$, and can be seen as:

$$
B(r,t) = \hat{B}(r,t) + \int_t^T Ce^{-\alpha s} \, ds
$$

We also know the bond pays at the risk free rate:

$$
dB = rBdt
$$

---

## PDE via Itô's Lemma

Using Itô's Lemma, we derive the PDE for the bond price $ B(r, t) $:

$$
\frac{\partial B}{\partial t} 
+ \kappa(\theta e^{\mu t} - r) \frac{\partial B}{\partial r}
+ \frac{1}{2} \sigma^2 r^{2\beta} \frac{\partial^2 B}{\partial r^2}
- rB + Ce^{-\alpha t} = 0
$$

This is subject to the boundary conditions:

$$
B(r, T; T) = F
$$

$$
\frac{\partial B}{\partial t} + \kappa \theta e^{\mu t} \frac{\partial B}{\partial r} + Ce^{-\alpha t} = 0 \quad \text{at } r = 0
$$

$$
B(r, t; T) \to 0 \quad \text{as } r \to \infty
$$


---
## Discretization Grid and Notation

We discretize the domain using a uniform grid:

- $r_j = j \Delta r$, for $j \in \mathbb{N}[0, j_{\text{max}}]$
- $t_i = i \Delta t$, for $i \in \mathbb{N}[0, i_{\text{max}}]$
- $B_j^i \approx B(r_j, t_i)$

We solve  backwards in time  from $i = i_{\text{max}} \rightarrow 0$ using the Crank-Nicolson method.

---

## Finite Difference Approximations

The finite difference approximations used are:

$$
\frac{\partial B}{\partial t} \approx \frac{B_j^{i+1} - B_j^i}{\Delta t}
$$

$$
\frac{\partial B}{\partial r} \approx \frac{1}{4 \Delta r} \left( B_{j+1}^{i+1} - B_{j-1}^{i+1} + B_{j+1}^i - B_{j-1}^i \right)
$$

$$
\frac{\partial^2 B}{\partial r^2} \approx \frac{1}{2 \Delta r^2} \left( B_{j+1}^{i+1} - 2 B_j^{i+1} + B_{j-1}^{i+1} + B_{j+1}^i - 2 B_j^i + B_{j-1}^i \right)
$$

$$
B(r_j, t_{i+1/2}) \approx \frac{1}{2}(B_j^i + B_j^{i+1})
$$

---

## Crank-Nicolson Scheme Structure

We rearrange the discretized PDE into the Crank-Nicolson matrix form:

$$
a_j B_{j-1}^i + b_j B_j^i + c_j B_{j+1}^i = d_j
$$

Where $a_j, b_j, c_j$ are the Crank-Nicolson coefficients evaluated at time step $i$, and $d_j$ is the right-hand side assembled from known values at $i+1$.

---

## Coefficient Definitions

Using the definitions from the  discretized PDE:

### Matrix Coefficients:

$$
a_j = \frac{\sigma^2 (j \Delta r)^{2\beta}}{4 (\Delta r)^2} - \frac{\kappa (\theta e^{\mu i \Delta t} - j \Delta r)}{4 \Delta r}
$$

$$
b_j = -\frac{1}{\Delta t} - \frac{1}{2} j \Delta r - \frac{\sigma^2 (j \Delta r)^{2\beta}}{2 (\Delta r)^2}
$$

$$
c_j = \frac{\sigma^2 (j \Delta r)^{2\beta}}{4 (\Delta r)^2} + \frac{\kappa (\theta e^{\mu i \Delta t} - j \Delta r)}{4 \Delta r}
$$

$$
d_j =
- \left[
\frac{\sigma^2 (j \Delta r)^{2\beta}}{4 (\Delta r)^2} - \frac{\kappa (\theta e^{\mu i \Delta t} - j \Delta r)}{4 \Delta r}
\right] B_{j-1}^{i+1}
$$

$$
- \left[
\frac{1}{\Delta t} + \frac{1}{2} j \Delta r + \frac{\sigma^2 (j \Delta r)^{2\beta}}{2 (\Delta r)^2}
\right] B_j^{i+1}
$$

$$
- \left[
\frac{\sigma^2 (j \Delta r)^{2\beta}}{4 (\Delta r)^2} + \frac{\kappa (\theta e^{\mu i \Delta t} - j \Delta r)}{4 \Delta r}
\right] B_{j+1}^{i+1}
- C e^{-\alpha i \Delta t}
$$


The PDE is solved on the domain $r \in [0, \infty)$ and $t < T$, with the following boundary and terminal conditions:

- Terminal condition (at maturity):
  $$
  B(r, t = T; T) = F
  $$

- Left boundary condition  (at $r = 0$):
  $$
  \frac{\partial B}{\partial t} + \kappa \theta e^{\mu t} \frac{\partial B}{\partial r} + C e^{-\alpha t} = 0
  $$

-  Right boundary condition  (as $r \to \infty$):
  $$
  B(r, t; T) \to 0
  $$

With the parameters given as:

- $T = 3$
- $F = 240$
- $\theta = 0.0289$
- $r_0 = 0.0238$
- $\kappa = 0.09389$
- $\mu = 0.0141$
- $C = 10.2$
- $\alpha = 0.01$
- $\beta = 0.418$
- $\sigma = 0.116$


---
We have selected $r_{max}$ as 1 in this problem

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [1]:
params = {
    'T': 3,
    'F': 240,
    'theta': 0.0289,
    'r0': 0.0238,
    'kappa': 0.09389,
    'mu': 0.0141,
    'C': 10.2,
    'alpha': 0.01,
    'beta': 0.418,
    'sigma': 0.116,
}

In [ ]:
class BondPDESolver:
    def __init__(self, params, i_max=100, j_max=100, r_max=1.0,left_boundary_fn=None,
                 right_boundary_fn=None):
        """
        The initialising function for the BondPDE class
        params: dictionary of parameters
        i_max: number of time steps
        j_max: number of r steps
        r_max: maximum value of r
        left_boundary_fn: function for left boundary condition
        right_boundary_fn: function for right boundary condition
        initial_condition_fn: function for initial condition
        """
        # Setup the parameters
        self.kappa = params['kappa']
        self.theta = params['theta']
        self.mu = params['mu']
        self.sigma = params['sigma']
        self.beta = params['beta']
        self.alpha = params['alpha']
        self.C = params['C']
        self.F = params['F']
        self.T = params['T']
        self.r0 = params['r0']

        # Set the grid parameters
        self.j_max = j_max
        self.i_max = i_max
        self.r_max = r_max
        self.dt =  self.T / self.i_max
        self.dr = self.r_max / self.j_max

        # Initialise the solution array B(r,t)
        self.B = np.zeros(( self.j_max + 1,self.i_max + 1)) # +1 due to zero indexing
        
        # Setup the boundary conditions
        self.left_boundary_fn = left_boundary_fn
        self.right_boundary_fn = right_boundary_fn

